# IMPORT DATA FROM IMBDb (rapidapi)

#### You can only make a query by movie title througth IMBb API.  For example "Titanic".
#### Movie title is mandatory if you want any type of query througth IMBb API.  For example "Titanic" --> https://imdb-api.com/API/Search/{APIKey}/Titanic.
#### You'll get Titanic movie info and also information about movies that matches close variants of that one, like "Titan". That can be an issue and a complicated situation in which we will have to extract from all the matches the one we want. If you make 500 requests from 500 different movies, thousand of other movies information will be there.
#### For this matter we can make a title query by title ID (unique for each movie) and will make sure we get only the information of the movies we want.  We will use a IMDb csv dataset already available in Kaggle which includes these ids and its correspodent movie title.
#### Note: two rounds of request were made because of the IMDb requests limits, only getting award information of 885 movies

#### We import out selected movie ids from our kaggle clean data 

In [4]:
import json
import pandas as pd
import json
import requests
from pandas import json_normalize
import os
from dotenv import load_dotenv
import time
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Read and convert the csv source data into a pandas dataframe.

# pd.set_option('display.max_columns', None)
movie_dataset_final_sample = pd.read_csv("movie_dataset_final_sample.csv")

In [3]:
kaggle_movie_ids = movie_dataset_final_sample["imdb_title_id"].tolist()
# kaggle_movie_ids

In [4]:
load_dotenv()

True

In [5]:
imdb_token = os.getenv("rapidapi_token")

## Gets Movie Ratings

In [6]:
def requestImdbRatings(url):
    request_count = 0
    if not imdb_token:
        raise ValueError("Necesitas un token, revisa el .env")


    querystring = {"tconst":f"{titleid}"}

    headers = {
    'x-rapidapi-host': "imdb8.p.rapidapi.com",
    'x-rapidapi-key': f"{imdb_token}"
    }
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    return response
    time.sleep(1)



In [31]:
movies_awards = []

for titleid in kaggle_movie_ids:

    url = "https://imdb8.p.rapidapi.com/title/get-awards-summary"

    movies_awards.append(requestImdbRatings(url))

In [32]:
print(movies_awards)

[{'@type': 'imdb.api.title.awardssummary', 'awardsSummary': {'otherNominationsCount': 2, 'otherWinsCount': 7}, 'id': '/title/tt0307913/', 'title': 'Our Father', 'titleType': 'movie', 'year': 2002}, {'@type': 'imdb.api.title.awardssummary', 'awardsSummary': {'otherNominationsCount': 19, 'otherWinsCount': 1}, 'id': '/title/tt1216475/', 'title': 'Cars 2', 'titleType': 'movie', 'year': 2011}, {'@type': 'imdb.api.title.awardssummary', 'awardsSummary': {'otherNominationsCount': 10, 'otherWinsCount': 6}, 'id': '/title/tt0247199/', 'title': 'The Believer', 'titleType': 'movie', 'year': 2001}, {'@type': 'imdb.api.title.awardssummary', 'awardsSummary': {'otherNominationsCount': 0, 'otherWinsCount': 2}, 'id': '/title/tt1605769/', 'title': 'The Monk', 'titleType': 'movie', 'year': 2011}, {'@type': 'imdb.api.title.awardssummary', 'awardsSummary': {'otherNominationsCount': 0, 'otherWinsCount': 0}, 'id': '/title/tt1182615/', 'title': 'Ah Long Pte Ltd', 'titleType': 'movie', 'year': 2008}, {'@type': '

In [33]:
movies_awards = pd.DataFrame(movies_awards)
movies_awards

,@type,awardsSummary,id,title,titleType,year,disambiguation,highlightedRanking
0,imdb.api.title.awardssummary,"{'otherNominationsCount': 2, 'otherWinsCount': 7}",/title/tt0307913/,Our Father,movie,2002,NaN,NaN
1,imdb.api.title.awardssummary,"{'otherNominationsCount': 19, 'otherWinsCount'...",/title/tt1216475/,Cars 2,movie,2011,NaN,NaN
2,imdb.api.title.awardssummary,"{'otherNominationsCount': 10, 'otherWinsCount'...",/title/tt0247199/,The Believer,movie,2001,NaN,NaN
3,imdb.api.title.awardssummary,"{'otherNominationsCount': 0, 'otherWinsCount': 2}",/title/tt1605769/,The Monk,movie,2011,NaN,NaN
4,imdb.api.title.awardssummary,"{'otherNominationsCount': 0, 'otherWinsCount': 0}",/title/tt1182615/,Ah Long Pte Ltd,movie,2008,NaN,NaN
...,...,...,...,...,...,...,...,...
495,imdb.api.title.awardssummary,"{'otherNominationsCount': 0, 'otherWinsCount': 0}",/title/tt5568330/,I am not Salvador,movie,2016,NaN,NaN
496,imdb.api.title.awardssummary,"{'otherNominationsCount': 0, 'otherWinsCount': 0}",/title/tt9101540/,Big Brother,movie,2020,I,NaN
497,imdb.api.title.awardssummary,"{'highlighted': {'awardName': 'Oscar', 'count'...",/title/tt0166924/,Mulholland Drive,movie,2001,NaN,NaN
498,imdb.api.title.awardssummary,"{'otherNominationsCount': 1, 'otherWinsCount': 1}",/title/tt1100119/,Shorts,movie,2008,NaN,NaN


In [39]:
# Get the info within awardsSummary dicts into columns

df_awardsSummary = json_normalize(movies_awards['awardsSummary'])
# df_awardsSummary.head()

In [40]:
# Concatenate columns of movies_awards and df_awardsSummary

df_awards_secondround = pd.concat([movies_awards, df_awardsSummary], axis=1)
# df_awards_secondround

In [ ]:
# df_awards_secondround.to_csv('df_awards_secondround.csv', sep =',',index = False)

#### We have already got 385 requests from another token so now we are going two sum the two rounds of requests into a new dataset of 885 movies

In [ ]:
# df_awards_secondround.to_csv('df_awards_secondround.csv', sep =',',index = False)

In [15]:
# df_awards_firstround = pd.read_csv("df_awards_firstround.csv")

In [45]:
df_awards_final = df_awards_firstround.append(df_awards_secondround)
df_awards_final.shape

(885, 14)

In [46]:
df_awards_final.head()

,@type,awardsSummary,id,title,titleType,year,disambiguation,highlightedRanking,otherNominationsCount,otherWinsCount,highlighted.awardName,highlighted.count,highlighted.eventId,highlighted.isWinner
0,imdb.api.title.awardssummary,"{'otherNominationsCount': 0, 'otherWinsCount': 0}",tt0242508,Harvard Man,movie,2001,NaN,NaN,0,0,NaN,NaN,NaN,NaN
1,imdb.api.title.awardssummary,"{'otherNominationsCount': 0, 'otherWinsCount': 0}",tt4080594,Yeob-gi-jeok-in geu-nyeo 2,movie,2016,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,imdb.api.title.awardssummary,"{'otherNominationsCount': 0, 'otherWinsCount': 0}",tt1294141,Dancing Ninja,movie,2010,NaN,NaN,0,0,NaN,NaN,NaN,NaN
3,imdb.api.title.awardssummary,"{'otherNominationsCount': 0, 'otherWinsCount': 0}",tt2088974,Vuonna 85,movie,2013,NaN,NaN,0,0,NaN,NaN,NaN,NaN
4,imdb.api.title.awardssummary,"{'otherNominationsCount': 3, 'otherWinsCount': 1}",tt6584902,Wallay,movie,2017,NaN,NaN,3,1,NaN,NaN,NaN,NaN


In [48]:
# Remove "/title/" from id column values

df_awards_final['id']= df_awards_final['id'].str.replace('/title/', '')

In [37]:
selected_columns = ['imdb_title_id', 'otherNominationsCount','otherWinsCount','highlighted.awardName','highlighted.count']

In [38]:
df_awards_final = pd.read_csv("df_awards_final.csv")
df_awards_final.head()

,imdb_title_id,title,year,otherNominationsCount,otherWinsCount,highlighted.awardName,highlighted.count
0,tt0242508,Harvard Man,2001,0,0,NaN,NaN
1,tt4080594,Yeob-gi-jeok-in geu-nyeo 2,2016,0,0,NaN,NaN
2,tt1294141,Dancing Ninja,2010,0,0,NaN,NaN
3,tt2088974,Vuonna 85,2013,0,0,NaN,NaN
4,tt6584902,Wallay,2017,3,1,NaN,NaN


In [39]:
df_awards_final = df_awards_final[df_awards_final.columns.intersection(selected_columns)]
df_awards_final.head()

,imdb_title_id,otherNominationsCount,otherWinsCount,highlighted.awardName,highlighted.count
0,tt0242508,0,0,NaN,NaN
1,tt4080594,0,0,NaN,NaN
2,tt1294141,0,0,NaN,NaN
3,tt2088974,0,0,NaN,NaN
4,tt6584902,3,1,NaN,NaN


In [26]:
# # Rename columns
df_awards_final.columns = ['imdb_title_id', 'otherNominationsCount','otherWinsCount','highlighted.awardName','highlighted.count']

In [40]:
# Export to csv
df_awards_final.to_csv('df_awards_final.csv', sep =',',index = False)